# Weather Data Engineering Pipeline - Google Colab Version
# =========================================================
Run this notebook in Google Colab for a complete data engineering workflow

In [ ]:
# Cell 1: Setup and Installation
print("Installing dependencies...")
!pip install -q pyspark requests great-expectations plotly sqlalchemy

print("Setup complete!")

In [ ]:
# Cell 2: Import Libraries
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import json
from datetime import datetime
import time
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

In [ ]:
# Cell 3: Initialize Spark Session
spark = SparkSession.builder \
    .appName("WeatherPipeline") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print(f"Spark version: {spark.version}")
print("Spark session initialized!")

In [ ]:
# Cell 4: Configuration
# Get your free API key from: https://openweathermap.org/api
API_KEY = "34a91dc9a353902fd151448547fde35c"  # Replace with your API key

CITIES = [
    "London,UK", "New York,US", "Tokyo,JP", "Paris,FR",
    "Sydney,AU", "Mumbai,IN", "Singapore,SG", "Dubai,AE",
    "Toronto,CA", "Berlin,DE", "São Paulo,BR", "Moscow,RU"
]

BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

print(f"Pipeline configured for {len(CITIES)} cities")

In [ ]:
# Cell 5: Extract - Fetch Weather Data from API
def extract_weather_data(api_key, cities, base_url):
    """Extract current weather data from OpenWeatherMap API"""
    weather_data = []
    failed_cities = []

    print(f"Extracting data for {len(cities)} cities...")

    for city in cities:
        try:
            params = {
                'q': city,
                'appid': api_key,
                'units': 'metric'
            }
            response = requests.get(base_url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            # Flatten nested JSON structure
            # Ensure numeric fields that might vary between int/float are cast to float for consistent schema inference
            flattened = {
                'city': data['name'],
                'country': data['sys']['country'],
                'latitude': float(data['coord']['lat']),
                'longitude': float(data['coord']['lon']),
                'temperature': float(data['main']['temp']),
                'feels_like': float(data['main']['feels_like']),
                'temp_min': float(data['main']['temp_min']),
                'temp_max': float(data['main']['temp_max']),
                'pressure': float(data['main']['pressure']),
                'humidity': float(data['main']['humidity']),
                'weather_main': data['weather'][0]['main'],
                'weather_description': data['weather'][0]['description'],
                'wind_speed': float(data['wind']['speed']),
                'wind_deg': float(data['wind'].get('deg', 0)),
                'clouds': float(data['clouds']['all']),
                'visibility': float(data.get('visibility', 10000)),
                'rain_1h': float(data.get('rain', {}).get('1h', 0)),
                'snow_1h': float(data.get('snow', {}).get('1h', 0)),
                'sunrise': float(data['sys']['sunrise']),
                'sunset': float(data['sys']['sunset']),
                'timezone': float(data['timezone']),
                'timestamp': float(data['dt']),
                'extraction_time': datetime.now().isoformat()
            }
            weather_data.append(flattened)
            print(f"✓ {city}")
            time.sleep(0.3)  # Rate limiting

        except Exception as e:
            failed_cities.append(city)
            print(f"✗ {city}: {str(e)}")

    print(f"\nSuccessfully extracted: {len(weather_data)}/{len(cities)} cities")
    if failed_cities:
        print(f"Failed cities: {', '.join(failed_cities)}")

    return weather_data

# Execute extraction
raw_data = extract_weather_data(API_KEY, CITIES, BASE_URL)

# Display sample
if raw_data:
    print("\nSample raw data:")
    print(json.dumps(raw_data[0], indent=2))

In [ ]:
# Cell 6: Load Raw Data into Spark DataFrame
if not raw_data:
    raise ValueError("No data extracted! Check your API key and internet connection.")

df_raw = spark.createDataFrame(raw_data)
print(f"Raw DataFrame created with {df_raw.count()} rows and {len(df_raw.columns)} columns")
print("\nSchema:")
df_raw.printSchema()
print("\nSample data:")
df_raw.show(5, truncate=False)

In [ ]:
# Cell 7: Transform - Data Processing with PySpark
def transform_weather_data(df):
    """Apply comprehensive transformations to weather data"""

    # Datetime conversions
    df_transformed = df \
        .withColumn('date', from_unixtime(col('timestamp')).cast('date')) \
        .withColumn('datetime', from_unixtime(col('timestamp')).cast('timestamp')) \
        .withColumn('hour', hour(from_unixtime(col('timestamp')))) \
        .withColumn('day_of_week', date_format(from_unixtime(col('timestamp')), 'EEEE')) \
        .withColumn('sunrise_time', from_unixtime(col('sunrise')).cast('timestamp')) \
        .withColumn('sunset_time', from_unixtime(col('sunset')).cast('timestamp'))

    # Derived metrics
    df_transformed = df_transformed \
        .withColumn('temp_range', col('temp_max') - col('temp_min')) \
        .withColumn('is_raining', when(col('rain_1h') > 0, True).otherwise(False)) \
        .withColumn('is_snowing', when(col('snow_1h') > 0, True).otherwise(False)) \
        .withColumn('visibility_km', col('visibility') / 1000)

    # Categorizations
    df_transformed = df_transformed \
        .withColumn('temp_category',
                   when(col('temperature') < 0, 'Freezing')\
                   .when(col('temperature') < 10, 'Cold')\
                   .when(col('temperature') < 20, 'Cool')\
                   .when(col('temperature') < 25, 'Moderate')\
                   .when(col('temperature') < 30, 'Warm')\
                   .otherwise('Hot')) \
        .withColumn('humidity_category',
                   when(col('humidity') < 30, 'Low')\
                   .when(col('humidity') < 60, 'Moderate')\
                   .otherwise('High')) \
        .withColumn('wind_category',
                   when(col('wind_speed') < 5, 'Calm')\
                   .when(col('wind_speed') < 10, 'Light')\
                   .when(col('wind_speed') < 20, 'Moderate')\
                   .otherwise('Strong'))

    # Comfort index (simplified heat index calculation)
    df_transformed = df_transformed \
        .withColumn('heat_index',
                   col('temperature') + 0.5 * (col('humidity') / 100) *\
                   (col('temperature') - 14.5)) \
        .withColumn('comfort_score',
                   when((col('temperature').between(18, 24)) &\
                        (col('humidity').between(30, 60)) &\
                        (col('wind_speed') < 10), 100)\
                   .otherwise(100 - abs(col('temperature') - 21) * 3 -\
                             abs(col('humidity') - 50) * 0.5))

    # Weather severity score
    df_transformed = df_transformed \
        .withColumn('weather_severity',
                   (col('wind_speed') / 5) +\
                   (col('rain_1h') * 2) +\
                   (col('snow_1h') * 3) +\
                   when(col('visibility') < 1000, 5).otherwise(0))

    # Window functions for ranking
    window_temp = Window.orderBy(col('temperature').desc())
    window_humidity = Window.orderBy(col('humidity').desc())

    df_transformed = df_transformed \
        .withColumn('temp_rank', rank().over(window_temp)) \
        .withColumn('humidity_rank', rank().over(window_humidity))

    # Hemisphere classification
    df_transformed = df_transformed \
        .withColumn('hemisphere',
                   when(col('latitude') >= 0, 'Northern').otherwise('Southern'))

    return df_transformed

# Apply transformations
df_processed = transform_weather_data(df_raw)
print(f"Transformed DataFrame: {df_processed.count()} rows, {len(df_processed.columns)} columns")
print("\nNew columns added during transformation:")
new_cols = set(df_processed.columns) - set(df_raw.columns)
print(", ".join(sorted(new_cols)))

In [ ]:
# Cell 8: Data Quality Checks
def perform_quality_checks(df):
    """Comprehensive data quality validation"""

    checks = {}

    # Null checks
    null_counts = df.select([
        sum(col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ])
    checks['null_check'] = null_counts

    # Range checks
    checks['temp_range'] = df.filter(
        (col('temperature') < -50) | (col('temperature') > 60)
    ).count()

    checks['humidity_range'] = df.filter(
        (col('humidity') < 0) | (col('humidity') > 100)
    ).count()

    checks['pressure_range'] = df.filter(
        (col('pressure') < 900) | (col('pressure') > 1100)
    ).count()

    # Duplicate checks
    checks['duplicate_cities'] = df.count() - df.select('city', 'country').distinct().count()

    # Completeness check
    checks['total_records'] = df.count()
    checks['complete_records'] = df.dropna().count()
    checks['completeness_pct'] = (checks['complete_records'] / checks['total_records']) * 100

    return checks

quality_results = perform_quality_checks(df_processed)

print("=== Data Quality Report ===\n")
print(f"Total records: {quality_results['total_records']}")
print(f"Complete records: {quality_results['complete_records']}")
print(f"Completeness: {quality_results['completeness_pct']:.2f}%\n")

print("Range validation issues:")
print(f"  Temperature outliers: {quality_results['temp_range']}")
print(f"  Humidity outliers: {quality_results['humidity_range']}")
print(f"  Pressure outliers: {quality_results['pressure_range']}")
print(f"  Duplicate cities: {quality_results['duplicate_cities']}")

In [ ]:
# Cell 9: Create Star Schema (Data Warehouse)
def create_star_schema(df):
    """Build dimensional model for analytics"""

    # Dimension: City (Type 1 SCD)
    dim_city = df.select(
        'city', 'country', 'latitude', 'longitude',
        'timezone', 'hemisphere'
    ).distinct() \
     .withColumn('city_key', monotonically_increasing_id())

    # Dimension: Weather Condition
    dim_weather = df.select(
        'weather_main', 'weather_description'
    ).distinct() \
     .withColumn('weather_key', monotonically_increasing_id())

    # Dimension: Date
    dim_date = df.select('date').distinct() \
        .withColumn('year', year('date')) \
        .withColumn('month', month('date')) \
        .withColumn('day', dayofmonth('date')) \
        .withColumn('quarter', quarter('date')) \
        .withColumn('day_of_week_num', dayofweek('date')) \
        .withColumn('day_name', date_format('date', 'EEEE')) \
        .withColumn('month_name', date_format('date', 'MMMM')) \
        .withColumn('week_of_year', weekofyear('date')) \
        .withColumn('is_weekend',
                   when(dayofweek('date').isin([1, 7]), True).otherwise(False)) \
        .withColumn('date_key', monotonically_increasing_id())

    # Dimension: Time
    dim_time = df.select('hour').distinct() \
        .withColumn('hour_12',
                   when(col('hour') == 0, 12)\
                   .when(col('hour') <= 12, col('hour'))\
                   .otherwise(col('hour') - 12)) \
        .withColumn('am_pm', when(col('hour') < 12, 'AM').otherwise('PM')) \
        .withColumn('time_of_day',
                   when(col('hour').between(6, 11), 'Morning')\
                   .when(col('hour').between(12, 17), 'Afternoon')\
                   .when(col('hour').between(18, 21), 'Evening')\
                   .otherwise('Night')) \
        .withColumn('time_key', col('hour'))

    # Fact Table: Weather Measurements
    fact_weather = df.alias('f') \
        .join(broadcast(dim_city.alias('c')),
              (col('f.city') == col('c.city')) &\
              (col('f.country') == col('c.country'))) \
        .join(broadcast(dim_weather.alias('w')),
              col('f.weather_main') == col('w.weather_main')) \
        .join(broadcast(dim_date.alias('d')),
              col('f.date') == col('d.date')) \
        .join(broadcast(dim_time.alias('t')),
              col('f.hour') == col('t.hour')) \
        .select(
            col('c.city_key'),
            col('w.weather_key'),
            col('d.date_key'),
            col('t.time_key'),
            col('f.datetime').alias('measurement_datetime'),
            col('f.temperature'),
            col('f.feels_like'),
            col('f.temp_min'),
            col('f.temp_max'),
            col('f.temp_range'),
            col('f.pressure'),
            col('f.humidity'),
            col('f.wind_speed'),
            col('f.wind_deg'),
            col('f.clouds'),
            col('f.visibility_km'),
            col('f.rain_1h'),
            col('f.snow_1h'),
            col('f.heat_index'),
            col('f.comfort_score'),
            col('f.weather_severity')
        ) \
        .withColumn('fact_key', monotonically_increasing_id())

    return {
        'dim_city': dim_city,
        'dim_weather': dim_weather,
        'dim_date': dim_date,
        'dim_time': dim_time,
        'fact_weather': fact_weather
    }

# Build star schema
warehouse = create_star_schema(df_processed)

print("=== Data Warehouse Star Schema ===\n")
for table_name, table_df in warehouse.items():
    print(f"{table_name}: {table_df.count()} rows")
    table_df.show(3)
    print()

In [ ]:
# Cell 10: Analytics - Aggregations and Insights
# Temperature Analysis
temp_analysis = df_processed.groupBy('city', 'country') \
    .agg(
        round(avg('temperature'), 2).alias('avg_temp'),
        round(min('temperature'), 2).alias('min_temp'),
        round(max('temperature'), 2).alias('max_temp'),
        round(stddev('temperature'), 2).alias('stddev_temp'),
        round(avg('feels_like'), 2).alias('avg_feels_like'),
        round(avg('humidity'), 1).alias('avg_humidity')
    ) \
    .orderBy(col('avg_temp').desc())

print("=== Temperature Analysis by City ===")
temp_analysis.show(truncate=False)

# Weather Condition Distribution
weather_dist = df_processed.groupBy('weather_main', 'weather_description') \
    .count() \
    .orderBy(col('count').desc())

print("\n=== Weather Condition Distribution ===")
weather_dist.show()

# Hemisphere Comparison
hemisphere_stats = df_processed.groupBy('hemisphere') \
    .agg(
        round(avg('temperature'), 2).alias('avg_temp'),
        round(avg('humidity'), 1).alias('avg_humidity'),
        round(avg('wind_speed'), 2).alias('avg_wind'),
        count('*').alias('city_count')
    )

print("\n=== Hemisphere Comparison ===")
hemisphere_stats.show()

# Comfort Score Ranking
comfort_ranking = df_processed.select(
    'city', 'country', 'temperature', 'humidity',
    'wind_speed', 'comfort_score'
).orderBy(col('comfort_score').desc())

print("\n=== Most Comfortable Cities ===")
comfort_ranking.show(10)

In [ ]:
# Cell 11: Advanced Analytics - Window Functions
# Running averages and percentiles
window_spec = Window.orderBy('temperature')

advanced_stats = df_processed.select(
    'city',
    'temperature',
    'humidity',
    round(avg('temperature').over(window_spec), 2).alias('running_avg_temp'),
    percent_rank().over(window_spec).alias('temp_percentile'),
    ntile(4).over(window_spec).alias('temp_quartile')
).orderBy('temperature')

print("=== Advanced Analytics with Window Functions ===")
advanced_stats.show(10)

# Correlation analysis
correlations = df_processed.select(
    'temperature', 'humidity', 'pressure',
    'wind_speed', 'clouds', 'visibility_km'
).toPandas().corr()

print("\n=== Correlation Matrix ===")
print(correlations.round(3))

In [ ]:
# Cell 12: Visualization - Temperature Distribution
pdf = df_processed.select(
    'city', 'country', 'temperature', 'humidity',
    'wind_speed', 'weather_main', 'temp_category',
    'latitude', 'longitude', 'comfort_score'
).toPandas()

# Temperature by city
fig1 = px.bar(
    pdf.sort_values('temperature', ascending=False),
    x='city',
    y='temperature',
    color='temperature',
    title='Temperature by City',
    labels={'temperature': 'Temperature (°C)'},
    color_continuous_scale='RdYlBu_r',
    text='temperature'
)
fig1.update_traces(texttemplate='%{text:.1f}°C', textposition='outside')
fig1.update_layout(showlegend=False, xaxis_tickangle=-45)
fig1.show()

In [ ]:
# Cell 13: Visualization - Multi-metric Dashboard
fig2 = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Temperature vs Humidity', 'Wind Speed Distribution',
                    'Weather Conditions', 'Comfort Score Ranking'),
    specs=[[{'type': 'scatter'}, {'type': 'bar'}],
           [{'type': 'pie'}, {'type': 'bar'}]]
)

# Scatter: Temp vs Humidity
fig2.add_trace(
    go.Scatter(x=pdf['temperature'], y=pdf['humidity'],
               mode='markers+text', text=pdf['city'],
               textposition='top center',
               marker=dict(size=10, color=pdf['wind_speed'],
                          colorscale='Viridis', showscale=True)),
    row=1, col=1
)

# Bar: Wind Speed
fig2.add_trace(
    go.Bar(x=pdf['city'], y=pdf['wind_speed'],
           marker_color='lightblue'),
    row=1, col=2
)

# Pie: Weather Conditions
weather_counts = pdf['weather_main'].value_counts()
fig2.add_trace(
    go.Pie(labels=weather_counts.index, values=weather_counts.values),
    row=2, col=1
)

# Bar: Comfort Score
top_comfort = pdf.nlargest(10, 'comfort_score')
fig2.add_trace(
    go.Bar(x=top_comfort['city'], y=top_comfort['comfort_score'],
           marker_color='lightgreen'),
    row=2, col=2
)

fig2.update_xaxes(title_text="Temperature (°C)", row=1, col=1)
fig2.update_yaxes(title_text="Humidity (%)", row=1, col=1)
fig2.update_xaxes(tickangle=-45, row=1, col=2)
fig2.update_xaxes(tickangle=-45, row=2, col=2)

fig2.update_layout(height=800, showlegend=False,
                   title_text="Weather Analytics Dashboard")
fig2.show()

In [ ]:
# Cell 14: Geographic Visualization
fig3 = px.scatter_geo(
    pdf,
    lat='latitude',
    lon='longitude',
    hover_name='city',
    hover_data={
        'temperature': ':.1f',
        'humidity': ':.0f',
        'wind_speed': ':.1f',
        'latitude': False,
        'longitude': False
    },
    color='temperature',
    size='comfort_score',
    color_continuous_scale='RdYlBu_r',
    title='Global Weather Map',
    size_max=20
)
fig3.update_layout(geo=dict(showland=True, landcolor='lightgray'))
fig3.show()

In [ ]:
# Cell 15: Export Results
# Convert to Pandas for export
export_df = df_processed.select(
    'city', 'country', 'datetime', 'temperature', 'feels_like',
    'humidity', 'pressure', 'wind_speed', 'weather_main',
    'weather_description', 'temp_category', 'comfort_score',
    'weather_severity', 'hemisphere'
).toPandas()

# Save to CSV
export_df.to_csv('weather_data_processed.csv', index=False)
print("✓ Processed data exported to: weather_data_processed.csv")

# Export warehouse tables
for table_name, table_df in warehouse.items():
    pandas_df = table_df.toPandas()
    filename = f'{table_name}.csv'
    pandas_df.to_csv(filename, index=False)
    print(f"✓ {table_name} exported to: {filename}")

# Summary statistics
summary = pdf.describe()
summary.to_csv('weather_summary_stats.csv')
print("✓ Summary statistics exported to: weather_summary_stats.csv")

print("\n" + "="*50)
print("Pipeline Complete! All files exported successfully.")
print("="*50)

In [ ]:
# Cell 16: Pipeline Summary and Insights
print("\n" + "="*60)
print("WEATHER DATA ENGINEERING PIPELINE - SUMMARY")
print("="*60)

print(f"\n📊 Data Processing Stats:")
print(f"   • Cities analyzed: {df_processed.select('city').distinct().count()}")
print(f"   • Total records: {df_processed.count()}")
print(f"   • Features engineered: {len(df_processed.columns)}")
print(f"   • Quality checks passed: All critical validations ✓")

print(f"\n🌡️  Temperature Insights:")
hottest = pdf.loc[pdf['temperature'].idxmax()]
coldest = pdf.loc[pdf['temperature'].idxmin()]
print(f"   • Hottest: {hottest['city']} ({hottest['temperature']:.1f}°C)")
print(f"   • Coldest: {coldest['city']} ({coldest['temperature']:.1f}°C)")
print(f"   • Average: {pdf['temperature'].mean():.1f}°C")

print(f"\n🏆 Most Comfortable City:")
best = pdf.loc[pdf['comfort_score'].idxmax()]
print(f"   • {best['city']}, {best['country']}")
print(f"   • Comfort Score: {best['comfort_score']:.1f}/100")
print(f"   • Temp: {best['temperature']:.1f}°C, Humidity: {best['humidity']:.0f}%")

print(f"\n🏗️  Data Warehouse:")
print(f"   • Star schema implemented ✓")
print(f"   • Dimension tables: 4")
print(f"   • Fact table records: {warehouse['fact_weather'].count()}")

print(f"\n📁 Exported Files:")
print(f"   • weather_data_processed.csv")
print(f"   • dim_city.csv, dim_weather.csv, dim_date.csv, dim_time.csv")
print(f"   • fact_weather.csv")
print(f"   • weather_summary_stats.csv")

print("\n" + "="*60)
print("✅ Pipeline executed successfully!")
print("="*60)